In [1]:
import os
import numpy as np
import pandas as pd
from pandas import read_csv
import geopandas as gpd
import fiona
import rasterio as rio
from tqdm import tqdm
import calendar
import datetime
import matplotlib.pyplot as plt
import plotly
import plotly.express as px


# 1: Stack all VIs in a folder then output CSV of extracted values for points

In [2]:
## stack ALL VIs in a folder. if return_csv is True - output points too 

## TO-DO:
## rasterio windowed writing 
## CRS check that reprojects polys to raster crs

def stack_extract_points(input_dir, out_name, inputShape, return_csv=True):
    
    rasList = []
    bandList = []
    
    for img in sorted(os.listdir(input_dir)):
        if img.endswith('.tif'):
            rasList.append(os.path.join(input_dir,img))
    print('number of images: ', len(rasList))
    band_paths=rasList
    # Read in metadata
    first_band = rio.open(band_paths[0], 'r')
    meta = first_band.meta.copy()
    # Replace metadata with new count and create a new file
    counts = 0
    for ifile in band_paths:
        with rio.open(ifile, 'r') as ff:
            counts += ff.meta['count']
    meta.update(count=counts)
    out_path = out_name + ".tif"
    with rio.open(out_path, 'w', **meta) as ff:
        for ii, ifile in tqdm(enumerate(band_paths)):
            bands = rio.open(ifile, 'r').read()
            if bands.ndim != 3:
                bands = bands[np.newaxis, ...]
            for band in bands:
                ff.descriptions = tuple([i[-11:-4] for i in band_paths])
                ff.write(band, ii+1)
            bandList.append(ff.descriptions)
            
    plots = []            
    multi_values_points = pd.Series()
    with fiona.open(inputShape) as shp:
        for feature in shp:
            siteID = feature['properties']['TARGET_FID']
            ProdID = feature['properties']['ID_Prod']
            Parcela = feature['properties']['Parcela']
            coords = feature['geometry']['coordinates']
            # Read pixel value at the given coordinates using Rasterio
            # NB: `sample()` returns an iterable of ndarrays.
            with rio.open(out_path) as stack_src:            
                value = [v for v in stack_src.sample([coords])] 
            # Update the pandas series accordingly
            multi_values_points.loc[siteID] = value
            plots.append(siteID)
            plots.append(ProdID)
            plots.append(Parcela)
            
    plots_arr = np.array(plots)
    plots_arr_reshape = plots_arr.reshape(int(plots_arr.shape[0]/3), -1)
    plots_df = pd.DataFrame(plots_arr_reshape, columns = ['FID','FieldID','Parcela'])
    df1 = pd.DataFrame(multi_values_points.values, index=multi_values_points.index)
    df1.columns = ['Val']
    df2 = pd.DataFrame(df1['Val'].explode())
    extracted_vals = pd.DataFrame(df2["Val"].to_list())
    extracted_vals.columns = bandList[0]
    extracted_vals['FID'] = np.arange(len(extracted_vals)).astype(str)
    extracted_plots = pd.merge(plots_df, extracted_vals, how = 'outer', on='FID')
    extracted_plots =  extracted_plots.set_index("FID")
    single_grid_vals = extracted_plots[~(extracted_plots.T == 0).any()]
    #print(single_grid_vals) # print if nothing is outputting, export as wide format
    ## REFORMAT
    df2 = single_grid_vals.set_index(["FieldID", "Parcela"])
    df2_T = df2.T
    df3 = df2_T.reset_index(level=0)
    df3["Julian"] = df3["index"].str.slice(start=0, stop=9).astype(str)
    df3["Date"] = pd.to_datetime(df3['Julian'], format='%Y%j')
    df4 = df3.set_index('Date')
    df5 = df4.drop(['index', 'Julian'], axis=1)
    df5 =  df5.reset_index(level=0) 
    df5_melt = pd.melt(df5, id_vars='Date')
    df5_melt.rename(columns = {'Date':'Date', 'ID_Prod': 'Code', 'Parcela': 'Parcela', 'value': 'Value'}, inplace=True)
    #print(df5_melt.to_string())

    if return_csv is True:
        out_csv_path = out_name + ".csv"
        print('exporting csv: ' + out_csv_path)
        return df5_melt.to_csv(out_csv_path)
    else:
        return('no csv for ' + out_csv_path)



In [3]:
new_grid_list = ['002306', '002307', '002386'] 

#######################################

for grid in new_grid_list: 
    for index in ['nbr']:
        input_dir = "/home/downspout-cel/crs_lc/raster/grids/" + grid + "/brdf_ts/ms/" + index + "/" ### crs_lc/wsa_lc for UTM/Albers
        out_name = "/home/downspout-cel/wsa_lc/TS/" + grid + "_" + index + "_full_TS"
        stack_extract_points(input_dir, out_name, inputShape='/home/sandbox-cel/wsa_lc/WSA_AOI/Plot_Locations/NI_ES_points_pt1_pt2_UTM16.shp', return_csv=True) ## change _ALBERS or UTM
        
        

number of images:  246


246it [03:01,  1.36it/s]
/home/lsharwood/.nasaenv/lib64/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/home/lsharwood/.nasaenv/lib64/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



exporting csv: /home/downspout-cel/wsa_lc/TS/002306_nbr_full_TS.csv
number of images:  246


246it [03:17,  1.24it/s]


exporting csv: /home/downspout-cel/wsa_lc/TS/002307_nbr_full_TS.csv
number of images:  246


246it [03:15,  1.26it/s]


exporting csv: /home/downspout-cel/wsa_lc/TS/002386_nbr_full_TS.csv


In [4]:
## stack ALL VIs in a folder. if return_csv is True - output points too 

## TO-DO:
## rasterio windowed writing 
## CRS check that reprojects polys to raster crs

def extract(in_rast, inputShape, return_csv=True):
    
    plots = []            
    multi_values_points = pd.Series()
    bandList = []
    with fiona.open(inputShape) as shp:
        for feature in shp:
            siteID = feature['properties']['TARGET_FID']
            ProdID = feature['properties']['oldIDPROD']
            Parcela = feature['properties']['Parcela']
            coords = feature['geometry']['coordinates']
            # Read pixel value at the given coordinates using Rasterio
            # NB: `sample()` returns an iterable of ndarrays.
            with rio.open(in_rast, 'r') as stack_src:            
                value = [v for v in stack_src.sample([coords])] 
                bandList.append(stack_src.descriptions)
            # Update the pandas series accordingly
            multi_values_points.loc[siteID] = value
            plots.append(siteID)
            plots.append(ProdID)
            plots.append(Parcela)
            
    plots_arr = np.array(plots)
    plots_arr_reshape = plots_arr.reshape(int(plots_arr.shape[0]/3), -1)
    plots_df = pd.DataFrame(plots_arr_reshape, columns = ['FID','FieldID','Parcela'])
    df1 = pd.DataFrame(multi_values_points.values, index=multi_values_points.index)
    df1.columns = ['Val']
    df2 = pd.DataFrame(df1['Val'].explode())
    extracted_vals = pd.DataFrame(df2["Val"].to_list())
    extracted_vals.columns = bandList[0]
    extracted_vals['FID'] = np.arange(len(extracted_vals)).astype(str)
    extracted_plots = pd.merge(plots_df, extracted_vals, how = 'outer', on='FID')
    extracted_plots =  extracted_plots.set_index("FID")
    single_grid_vals = extracted_plots[~(extracted_plots.T == 0).any()]
    #print(single_grid_vals) # print if nothing is outputting, export as wide format
    ## REFORMAT
    df2 = single_grid_vals.set_index(["FieldID", "Parcela"])
    df2_T = df2.T
    df3 = df2_T.reset_index(level=0)
    df3["Julian"] = df3["index"].str.slice(start=0, stop=9).astype(str)
    df3["Date"] = pd.to_datetime(df3['Julian'], format='%Y%j')
    df4 = df3.set_index('Date')
    df5 = df4.drop(['index', 'Julian'], axis=1)
    df5 =  df5.reset_index(level=0) 
    df5_melt = pd.melt(df5, id_vars='Date')
    df5_melt.rename(columns = {'Date':'Date', 'ID_Prod': 'Code', 'Parcela': 'Parcela', 'value': 'Value'}, inplace=True)
    #print(df5_melt.to_string())
    if return_csv is True:
        out_csv_path = in_rast[:-4] + "_new.csv"
        print('exporting csv: ' + out_csv_path)
        return df5_melt.to_csv(out_csv_path)
    else:
        return('no csv for ' + out_csv_path)



In [5]:
grid_list_ALBERS = ['001892','001972','002047','002056','002127','002225','002387','002550']
grid_list_UTM = ['002306','002307','002386','002388','002548','002549'] 


#######################################

for grid in grid_list_UTM: ## change _ALBERS or UTM
    for index in ['nbr']:
        in_rast = "/home/downspout-cel/wsa_lc/TS/" + grid + "_" + index + "_full_TS.tif"
        extract(in_rast, inputShape='/home/sandbox-cel/wsa_lc/WSA_AOI/Plot_Locations/NI_ES_points_pt1_pt2_UTM16.shp', return_csv=True) ## change _ALBERS or UTM
        
        
for grid in grid_list_ALBERS: ## change _ALBERS or UTM
    for index in ['nbr']:
        in_rast = "/home/downspout-cel/wsa_lc/TS/" + grid + "_" + index + "_full_TS.tif"
        extract(in_rast, inputShape='/home/sandbox-cel/wsa_lc/WSA_AOI/Plot_Locations/NI_ES_points_pt1_pt2_aea.shp', return_csv=True) ## change _ALBERS or UTM
        
               
    

/home/lsharwood/.nasaenv/lib64/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



exporting csv: /home/downspout-cel/wsa_lc/TS/002306_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002307_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002386_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002388_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002548_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002549_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/001892_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/001972_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002047_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002056_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002127_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002225_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002387_nbr_full_TS_new.csv
exporting csv: /home/downspout-cel/wsa_lc/TS/002550_nbr_full_TS_

# 2: Plot temporal profiles

In [6]:
def temporal_profile(input_dir, grid, index_list):
    for i in index_list:
        input_csv =  input_dir + grid + "_" + i + "_full_TS_new.csv"
        df = pd.read_csv(input_csv, header=0, index_col=0, parse_dates=True)
        fig = px.line(df, x='Date', y='Value', title= i.upper() + ' Time Series (for plots in grid ' + grid + ")", 
                      color='FieldID', 
                      color_discrete_sequence=px.colors.qualitative.Dark2, # Bold, Vivid, Dark2, Pastel
                      line_dash='Parcela')
        fig.update_layout(xaxis_title='Date', yaxis_title='VI Value')
        fig.update_xaxes(rangeslider_visible=True, 
                         rangeselector=dict(buttons=list([dict(count=6, label="6m", step="month", stepmode="backward"),
                                                          dict(count=1, label="1y", step="year", stepmode="backward"), 
                                                          dict(count=2, label="2y", step="year", stepmode="backward"), 
                                                          dict(step="all")]) ))
        fig.show()


In [8]:
NI_grids = ['002225','002306', '002307', '002386', '002387','002388','002548','002549','002550']
ES_grids = ['001892','001972','002047','002056','002127']
indices = ['evi2', 'kndvi', 'nbr'] #['evi2', 'kndvi', 'gcvi', 'nbr']
best = ['002225','002387','002388','002548','002549']

#####################################

for i in ['002387']:
    temporal_profile(input_dir="/home/downspout-cel/wsa_lc/TS/", grid=i, index_list=['evi2'])
    practices_df = pd.read_csv("/home/downspout-cel/wsa_lc/TS/practices_NI_ES.csv", dtype=str) 
    grid_practice_df = practices_df[practices_df['Grid'] == str(i)]
    practices = pd.pivot(grid_practice_df, index=['ID_Prod','Parcela', 'Nom.Cob'], columns=['Temporada','Ano'], values=['Cultivo'])
    display(practices)

Cultivo                                 \
Temporada                             Postrera Primera Postrera Primera         
Ano                                       2016    2016     2017    2017  2018   
ID_Prod Parcela Nom.Cob                                                         
NI_248  ASA     maíz-canavalia             NaN     NaN      NaN     NaN   NaN   
                maíz-frijol-canavalia   Frijol    Maiz   Frijol    Maiz  Maiz   
        Testigo no                      Frijol    Maiz   Frijol    Maiz  Maiz   
NI_258  ASA     maíz-canavalia             NaN     NaN      NaN     NaN   NaN   
                maíz-frijol-canavalia   Frijol    Maiz   Frijol    Maiz  Maiz   
        Testigo no                      Frijol    Maiz   Frijol    Maiz  Maiz   

                                                      
Temporada                                   Postrera  
Ano                                    2019     2019  
ID_Prod Parcela Nom.Cob                               
NI_248  ASA     maíz-canavalia          NaN     None  
                maíz-frijol-canavalia  Maiz      NaN  
        Testigo no                     Maiz     None  
NI_258  ASA     maíz-canavalia          NaN     None  
                maíz-frijol-canavalia  Maiz      NaN  
        Testigo no                     Maiz     None

In [ ]:
!jupyter nbconvert --to html CRS_stack_plot_TS.ipynb